In [1]:
import sys
import os
import py_entitymatching as em
print('magellan version:' + em.__version__)
import re
import csv
import pandas as pd
import math

from cleaner import Cleaner
from constants import csv_headers

magellan version:0.1.0


In [2]:
working_dir = os.path.dirname(os.getcwd())
path_to_csv_dir = working_dir + os.sep + 'csv_files'+ os.sep

# STEP 1 - PRE-PROCESSING DATA

In this stage, we need to preprocess data before applying Megellan. This is because our datasets (especially the AOM dataset) are quite dirty, and therefore adversely affecting Megellan's blocking and matching functions. For example, states can take any value of "CA", "California", or "CA - California".

In this step, we will clean the following variables:
* Country name (e.g. Whed data has 2 Belgiums: (1) Belgium - French Community and (2) Belgium - Flemish Community)
* State name
* City name
* Affiliation name
* Email server domain (we will only capture the university information from the email server domain - if there is any)


### 1.A. Clean AOM data

In [4]:
aom_cleaner = Cleaner(path_to_csv_dir + '_aom.csv', csv_headers.AOM_INDEX, csv_headers.AOM)

aom_cleaner.clean_affiliation('a_name')
aom_cleaner.clean_email_server('a_email_server')
aom_cleaner.clean_city('a_city')
aom_cleaner.clean_country('a_country')
aom_cleaner.clean_states('a_prov', 'a_country')

aom_cleaner.to_csv(path_to_csv_dir + '_aom_cleaned.csv')
aom_cleaner.data.head(n=3)

,a_name,a_city,a_prov,a_country,a_email_server
person_id,,,,,
1,academy management,briarcliff manor,new york,united states,aom
4,northeastern university,boston,massachusetts,united states,gmail
5,skidmore college,saratoga springs,new york,united states,skidmore


### 1.B. Clean WHED data

In [3]:
whed_cleaner = Cleaner(path_to_csv_dir + '_whed.csv', csv_headers.WHED_INDEX, csv_headers.WHED, encoding = 'ISO-8859-1')

whed_cleaner.clean_affiliation('a_name')
whed_cleaner.clean_city('a_city')
whed_cleaner.clean_country('a_country')

whed_cleaner.to_csv(path_to_csv_dir + '_whed_cleaned.csv')
whed_cleaner.data.head(n=3)

,a_name,a_country,a_city,a_prov,a_web
a_id,,,,,
2,pampanga state agricultural university,philippines,magalang,Pampanga,http://www.pac.edu.ph
4,les roches international school hotel management,switzerland,bluche crans montana,Bluche-Crans-Montana,http://www.lesroches.edu
6,dharma gate budapest buddhist university,hungary,budapest,NaN,http://www.tkbf.eu


# STEP 2 - MAGELLAN - BLOCKING

In [3]:

AOM = em.read_csv_metadata(path_to_csv_dir + '_aom_cleaned.csv', key = csv_headers.AOM_INDEX)
print(em.get_key(AOM))

WHED = em.read_csv_metadata(path_to_csv_dir + '_whed_cleaned.csv', key = csv_headers.WHED_INDEX)
print(em.get_key(WHED))


Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


person_id
a_id


In [21]:
# building inverted index based on B
sample_WHED, sample_AOM = em.down_sample(WHED, AOM, size=3000, y_param=10)
# sample_WHED, sample_AOM = em.down_sample(WHED, AOM, size=300, y_param=10)
print(len(sample_WHED))
print(len(sample_AOM))
em.show_properties(sample_WHED)
em.show_properties(sample_AOM)

0%                          100%
[##############################] | ETA: 00:00:14 | ETA: 00:00:14 | ETA: 00:00:14 | ETA: 00:00:13 | ETA: 00:00:13 | ETA: 00:00:12 | ETA: 00:00:12 | ETA: 00:00:11 | ETA: 00:00:11 | ETA: 00:00:10 | ETA: 00:00:10 | ETA: 00:00:09 | ETA: 00:00:09 | ETA: 00:00:08 | ETA: 00:00:08 | ETA: 00:00:07 | ETA: 00:00:06 | ETA: 00:00:06 | ETA: 00:00:05 | ETA: 00:00:05 | ETA: 00:00:04 | ETA: 00:00:04 | ETA: 00:00:03 | ETA: 00:00:03 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00

11231
3000
id: 4653972056
key: a_id
id: 4654349672
key: person_id



Total time elapsed: 00:00:15


In [22]:
def match_country_prov(ltuple, rtuple):
#   return True  - if the intention is to drop the pair
#          False - if the intention is to keep the tuple pair
    l_country = ltuple['a_country']
    r_country = rtuple['a_country']
    l_prov = ltuple['a_prov']
    r_prov = rtuple['a_prov']
    
    if l_country != r_country:
        return True
    elif l_country == "united states":
        if l_prov != r_prov:
            return True
        else:
            return False
    else:
        return False

def match_overlap(ltuple, rtuple):
    # There are cases where the AOM does not have affliation name
    # This does not happen with WHED
    if pd.isnull(ltuple['a_name']) or pd.isnull(rtuple['a_name']):
        return True
    
    l_name = ltuple['a_name']
    r_name = rtuple['a_name']
    
    l_name = re.sub(r"(university|school|institute)", "", l_name).strip()
    r_name = re.sub(r"(university|school|institute)", "", r_name).strip()
    
    l_tokens = em.tok_wspace(ltuple['a_name'])
    r_tokens = em.tok_wspace(rtuple['a_name'])
    
    if len(l_tokens) > 1 and len(r_tokens) > 1:
        return em.overlap_coeff(l_tokens, r_tokens) < 1
    else:
        l_tokens = em.get_tokenizers_for_blocking()['qgm_3'](ltuple['a_name'])
        r_tokens = em.get_tokenizers_for_blocking()['qgm_3'](rtuple['a_name'])
        return em.overlap_coeff(l_tokens, r_tokens) < 0.8
    
def match_combined(ltuple, rtuple):
    if match_country_prov(ltuple, rtuple):
        return True
    else:
        return match_overlap(ltuple, rtuple)
    
def blocking(A, B, A_headers, B_headers):
    bb = em.BlackBoxBlocker()
    bb.set_black_box_function(match_combined)
    C = bb.block_tables(A, B, l_output_attrs=A_headers, r_output_attrs=B_headers)
    return C

C = blocking(sample_WHED, sample_AOM, csv_headers.WHED, csv_headers.AOM)


0%                          100%
[##############################] | ETA: 00:21:39 | ETA: 00:20:48 | ETA: 00:20:02 | ETA: 00:19:16 | ETA: 00:18:32 | ETA: 00:17:49 | ETA: 00:17:04 | ETA: 00:16:21 | ETA: 00:15:39 | ETA: 00:14:54 | ETA: 00:14:08 | ETA: 00:13:22 | ETA: 00:12:38 | ETA: 00:11:53 | ETA: 00:11:09 | ETA: 00:10:24 | ETA: 00:09:40 | ETA: 00:08:56 | ETA: 00:08:12 | ETA: 00:07:27 | ETA: 00:06:42 | ETA: 00:05:58 | ETA: 00:05:14 | ETA: 00:04:29 | ETA: 00:03:45 | ETA: 00:03:01 | ETA: 00:02:15 | ETA: 00:01:30 | ETA: 00:00:45 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:22:43


In [23]:
C.head(n = 50000)


,_id,ltable_a_id,rtable_person_id,ltable_a_name,ltable_a_city,ltable_a_prov,ltable_a_country,ltable_a_web,rtable_a_name,rtable_a_city,rtable_a_prov,rtable_a_country,rtable_a_email_server
0,0,9,54203,aalborg university,aalborg,NaN,denmark,http://www.aau.dk,aalborg university,aalborg,NaN,denmark,aau
1,1,9,54111,aalborg university,aalborg,NaN,denmark,http://www.aau.dk,aalborg university,branbrand,NaN,denmark,u-k
2,2,10,9489,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,helsinki,NaN,finland,aalto
3,3,10,6105,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,espoo,NaN,finland,aalto
4,4,10,52765,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,espoo,finland,finland,aalto
5,5,10,50927,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,helsinki,NaN,finland,aalto
6,6,10,38224,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,espoo,NaN,finland,aalto
7,7,10,58343,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,helsinki,NaN,finland,aalto
8,8,10,47260,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,helsinki,NaN,finland,aalto
9,9,10,44615,aalto university,espoo,NaN,finland,http://www.aalto.fi/fi/,aalto university,helsinki,NaN,finland,aalto


In [24]:
C.size

30823

In [25]:

em.save_table(C, path_to_csv_dir + 'matching_pairs_table.pkl')

File already exists at /Users/carepjan/code/website/stage3/csv_files/matching_pairs_table.pkl; Overwriting it
Metadata file already exists at /Users/carepjan/code/website/stage3/csv_files/matching_pairs_table.pklmetadata. Overwriting it


True

In [26]:
em.to_csv_metadata(C, path_to_csv_dir + 'matching_pairs_metadata.csv')

File already exists at /Users/carepjan/code/website/stage3/csv_files/matching_pairs_metadata.csv; Overwriting it
Metadata file already exists at /Users/carepjan/code/website/stage3/csv_files/matching_pairs_metadata.metadata. Overwriting it


True

In [12]:
S = em.sample_table(C, 40)

In [40]:
sample_WHED.set_index([csv_headers.WHED_INDEX])
print(em.get_key(sample_WHED))

a_id


In [41]:
print(em.get_key(sample_AOM))

person_id


In [44]:
print(em.show_properties(sample_AOM))
print(em.show_properties(C))

id: 4673743000
key: person_id
None
id: 4706378920
key: _id
fk_ltable: ltable_a_id
fk_rtable: rtable_person_id
ltable(obj.id): 4560893432
rtable(obj.id): 4673743000
None


In [46]:
print(em.show_properties(C))

id: 4706378920
key: _id
fk_ltable: ltable_a_id
fk_rtable: rtable_person_id
ltable(obj.id): 4560893432
rtable(obj.id): 4673743000
None


In [51]:
sample_AOM.to_csv(path_to_csv_dir + 'sample_AOM.csv', encoding = 'UTF-8', index_label = csv_headers.AOM_INDEX)
sample_WHED.to_csv(path_to_csv_dir + 'sample_WHED.csv', encoding = 'UTF-8', index_label = csv_headers.WHED_INDEX)